In [28]:
%matplotlib widget
# %matplotlib inline
%load_ext line_profiler

import numpy as np
import seaborn as sns

from scipy import spatial, stats
from functools import reduce
from tqdm import tqdm
import matplotlib.pyplot as plt
from pprint import pprint
import itertools
import cupy as cp
from cupyimg.scipy.ndimage.morphology import binary_hit_or_miss

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [29]:
def are_equal(kernel1, kernel2) -> bool:
    return (kernel1 == kernel2).all()

In [30]:
def get_unique_orientations(cube):
    """
    List all possible unique variations of the given 3d array involving both rotation and reflection
    Inspired by @Colonel Panic at https://stackoverflow.com/questions/33190042/how-to-calculate-all-24-rotations-of-3d-array
    """
    assert np.unique(cube, return_counts=True)[1][1] == 4

    variations = []

    def rotations4(cube, axes):
        """List the four rotations of the given 3d array in the plane spanned by the given axes."""
        for i in range(4):
            variations.append(np.rot90(cube, i, axes))

    for arr in [cube, np.flip(cube, axis=0), np.flip(cube, axis=1), np.flip(cube, axis=2)]:
    # for arr in [cube]:
        # imagine shape is pointing in axis 0 (up)
        # 4 rotations about axis 0
        rotations4(arr, (1,2))

        # rotate 180 about axis 1, now shape is pointing down in axis 0
        # 4 rotations about axis 0
        rotations4(np.rot90(arr, 2, axes=(0,2)), (1,2))

        # rotate 90 or 270 about axis 1, now shape is pointing in axis 2
        # 8 rotations about axis 2
        rotations4(np.rot90(arr, 1, axes=(0,2)), (0,1))
        rotations4(np.rot90(arr, -1, axes=(0,2)), (0,1))

        # rotate about axis 2, now shape is pointing in axis 1
        # 8 rotations about axis 1
        rotations4(np.rot90(arr, 1, axes=(0,1)), (0,2))
        rotations4(np.rot90(arr, -1, axes=(0,1)), (0,2))

    return np.unique(variations, axis=0)

In [31]:
def check_for_duplicates(a, b):
    """
    Returns True if there are any duplicates
    """
    combinations = [p for p in itertools.product(a, b)]
    return any([np.array_equal(i, j) for i, j in combinations])

In [32]:
kernels = []

# Format: [[front],
#          [middle],
#          [back]]

# Filter 1
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0],
                         ],
                         [[0, 1, 0],
                          [1, 1, 1],
                          [0, 0, 0],
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0],]]))

# Filter 2
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 1],
                          [0, 0, 0]
                         ],
                         [[0, 1, 0],
                          [1, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 3
kernels.append(np.array([[[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 1, 0],
                          [1, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 4
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[0, 1, 0],
                          [1, 1, 0],
                          [0, 0, 1]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 5
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 1]
                         ],
                         [[0, 1, 0],
                          [1, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 6
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [1, 0, 0]
                         ],
                         [[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 7
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [1, 0, 0]]]))

# Filter 8
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 1, 0]
                         ],
                         [[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 9
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 1]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))
                                     
# Filter 10
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 1]]]))

# Filter 11
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 1]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 12
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 1, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 13
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 14
kernels.append(np.array([[[0, 1, 0],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 15
kernels.append(np.array([[[0, 0, 0],
                          [1, 0, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 16
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 1, 0],
                          [0, 1, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 17
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 1, 0]
                         ],
                         [[1, 0, 1],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 18
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 1]
                         ],
                         [[1, 0, 1],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 19
kernels.append(np.array([[[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 1],
                          [0, 1, 0],
                          [1, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 20
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [0, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 1, 0]
                         ],
                         [[1, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

# Filter 21
kernels.append(np.array([[[0, 0, 1],
                          [0, 0, 0],
                          [1, 0, 0]
                         ],
                         [[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]
                         ],
                         [[1, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]]]))

In [33]:
### Uncomment to check for duplicate filters

# for a, b in itertools.combinations(kernels, 2):
#     a_orientations = get_unique_orientations(a)
#     b_orientations = get_unique_orientations(b)

#     if check_for_duplicates(a_orientations, b_orientations):
#         raise ValueError("Duplicate Filter Found!")

In [34]:
unique_kernels = []

for kernel in kernels:
    unique_kernels.extend(get_unique_orientations(kernel))

In [37]:
image = cp.zeros((76, 1500, 3000), dtype=cp.uint16)
image[30, 600, 2000] = 1
image[31, 600, 2000] = 1
image[30, 601, 2000] = 1
image[30, 599, 2000] = 1

branch_pts_img = cp.zeros(image.shape, dtype=int)

for kernel in tqdm(unique_kernels):
    kernel = cp.asarray(kernel)
    branch_pts_img = cp.logical_or(binary_hit_or_miss(image, structure1=kernel),
                                   branch_pts_img)
branch_pts_img = branch_pts_img.astype(np.uint8) * 255

100%|█████████████████████████████████████████| 520/520 [00:15<00:00, 32.72it/s]


In [36]:
unique, counts = cp.unique(branch_pts_img, return_counts=True)
print(unique)
print(counts)

[False  True]
[341999999         1]


In [39]:
n = (1,2,3)
m = n + (4,)
m

(1, 2, 3, 4)